# Алгоритмы поиска кратчайших путей в графах


## Введение


**Определение:** Задача поиска кратчайшего пути (Shortest Path Problem)
Вход: Взвешенный граф $G = (V, E)$ с весовой функцией $w: E \rightarrow \mathbb{R}$ и выделенная вершина $s \in V$.
Выход: Для каждой вершины $v \in V$ длина кратчайшего пути от $s$ до $v$.

**Определение:** Кратчайший путь (Shortest Path) из вершины $s$ в вершину $t$ — это путь $p = \langle v_0, v_1, \ldots, v_k \rangle$, где $v_0 = s$, $v_k = t$, и для любого другого пути $p'$ из $s$ в $t$ выполняется $w(p) \leq w(p')$, где $w(p) = \sum_{i=1}^{k} w(v_{i-1}, v_i)$.

**Определение:** Длина кратчайшего пути (Shortest Path Distance) $\delta(s, t)$ из вершины $s$ в вершину $t$ — это вес кратчайшего пути из $s$ в $t$. Если пути из $s$ в $t$ не существует, то $\delta(s, t) = \infty$.

**Определение:** Отрицательный цикл (Negative Cycle) — это цикл в графе, сумма весов рёбер которого отрицательна.

**Замечание:** В графе с отрицательными циклами задача поиска кратчайшего пути может не иметь решения, так как длина кратчайшего пути может быть неограниченно малой.


## Алгоритм Дейкстры


**Идея:** Алгоритм Дейкстры решает задачу поиска кратчайших путей из одной вершины во все остальные для графа с неотрицательными весами рёбер. Алгоритм использует жадную стратегию, на каждом шаге выбирая вершину с наименьшей текущей оценкой расстояния.

**Теорема:** Алгоритм Дейкстры корректно находит кратчайшие пути от исходной вершины до всех достижимых вершин в графе с неотрицательными весами рёбер.

$\square$
Докажем корректность алгоритма Дейкстры с помощью индукции по количеству обработанных вершин.

База индукции: После инициализации для исходной вершины $s$ значение $d[s] = 0$ является корректным, так как кратчайший путь из $s$ в $s$ имеет нулевую длину.

Индукционный переход: Предположим, что после обработки $k$ вершин для каждой обработанной вершины $u$ значение $d[u]$ равно длине кратчайшего пути от $s$ до $u$.

Рассмотрим $(k+1)$-ю обрабатываемую вершину $v$. Докажем, что $d[v] = \delta(s, v)$.

Предположим противное: $d[v] > \delta(s, v)$. Тогда существует путь $p = \langle s, v_1, v_2, \ldots, v_m, v \rangle$ такой, что $w(p) < d[v]$.

Пусть $v_i$ — первая вершина на пути $p$, которая ещё не была обработана алгоритмом. Тогда $v_{i-1}$ уже обработана, и по индукционному предположению $d[v_{i-1}] = \delta(s, v_{i-1})$.

При обработке $v_{i-1}$ алгоритм должен был обновить значение $d[v_i]$ до $d[v_{i-1}] + w(v_{i-1}, v_i)$, что не превышает $\delta(s, v_i)$.

Поскольку алгоритм выбирает вершину с минимальным значением $d$ среди необработанных, и $d[v_i] \leq \delta(s, v_i) \leq \delta(s, v) < d[v]$, то вершина $v_i$ должна была быть выбрана раньше $v$. Противоречие.

Следовательно, $d[v] = \delta(s, v)$.
$\blacksquare$


In [ ]:
import heapq
from typing import Dict, List, Tuple, Set

def dijkstra(graph: Dict[int, List[Tuple[int, int]]], start: int) -> Dict[int, int]:
    """
    Реализация алгоритма Дейкстры для поиска кратчайших путей от начальной вершины.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, вес ребра)
        start: начальная вершина

    Returns:
        словарь расстояний от начальной вершины до всех достижимых вершин
    """
    # Инициализация расстояний
    distances = {vertex: float('infinity') for vertex in graph}
    distances[start] = 0

    # Очередь с приоритетами для выбора вершины с минимальным расстоянием
    priority_queue = [(0, start)]

    # Множество посещенных вершин
    visited: Set[int] = set()

    while priority_queue:
        # Извлекаем вершину с минимальным расстоянием
        current_distance, current_vertex = heapq.heappop(priority_queue)

        # Если вершина уже обработана, пропускаем
        if current_vertex in visited:
            continue

        # Помечаем вершину как посещенную
        visited.add(current_vertex)

        # Если текущее расстояние больше уже найденного, пропускаем
        if current_distance > distances[current_vertex]:
            continue

        # Обновляем расстояния до соседних вершин
        for neighbor, weight in graph[current_vertex]:
            distance = current_distance + weight

            # Если найден более короткий путь, обновляем расстояние
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances

# Пример использования
graph = {
    0: [(1, 4), (2, 1)],
    1: [(3, 1)],
    2: [(1, 2), (3, 5)],
    3: []
}

distances = dijkstra(graph, 0)
print(distances)  # {0: 0, 1: 3, 2: 1, 3: 4}


**Теорема:** Временная сложность алгоритма Дейкстры составляет $O((|V| + |E|) \log |V|)$ при использовании бинарной кучи и $O(|V|^2)$ при использовании массива.

$\square$
При использовании бинарной кучи:
- Извлечение минимума из кучи выполняется за $O(\log |V|)$
- Каждая вершина извлекается из кучи не более одного раза: $O(|V| \log |V|)$
- Каждое ребро просматривается не более одного раза, и для каждого может потребоваться операция уменьшения ключа в куче: $O(|E| \log |V|)$

Итоговая сложность: $O(|V| \log |V| + |E| \log |V|) = O((|V| + |E|) \log |V|)$

При использовании массива:
- Поиск вершины с минимальным расстоянием требует $O(|V|)$ операций
- Этот поиск выполняется $|V|$ раз: $O(|V|^2)$
- Обновление расстояний для всех рёбер требует $O(|E|)$ операций

Итоговая сложность: $O(|V|^2 + |E|) = O(|V|^2)$ (так как в худшем случае $|E| = O(|V|^2)$)
$\blacksquare$


**Замечание:** Алгоритм Дейкстры не работает корректно для графов с отрицательными весами рёбер. Это связано с тем, что жадная стратегия выбора вершины с минимальным текущим расстоянием может не учитывать возможность уменьшения расстояния через отрицательные рёбра.


## Алгоритм Беллмана-Форда


**Идея:** Алгоритм Беллмана-Форда решает задачу поиска кратчайших путей из одной вершины во все остальные для графа с произвольными весами рёбер, включая отрицательные. Алгоритм также может определить наличие отрицательного цикла, достижимого из исходной вершины.

**Теорема:** Алгоритм Беллмана-Форда корректно находит кратчайшие пути от исходной вершины до всех достижимых вершин в графе с произвольными весами рёбер, если в графе нет отрицательных циклов, достижимых из исходной вершины.

$\square$
Докажем корректность алгоритма Беллмана-Форда.

Пусть $d[v]$ — оценка расстояния от исходной вершины $s$ до вершины $v$, вычисленная алгоритмом после $i$ итераций внешнего цикла.

Утверждение: После $i$-й итерации внешнего цикла $d[v]$ равно длине кратчайшего пути от $s$ до $v$, содержащего не более $i$ рёбер.

Докажем это утверждение индукцией по $i$.

База индукции ($i = 0$): После инициализации $d[s] = 0$ и $d[v] = \infty$ для всех $v \neq s$. Это корректно, так как путь длины 0 существует только до самой вершины $s$.

Индукционный переход: Предположим, что утверждение верно для $(i-1)$-й итерации. Докажем, что оно верно для $i$-й итерации.

Рассмотрим произвольную вершину $v$ и кратчайший путь $p$ от $s$ до $v$, содержащий не более $i$ рёбер. Если $p$ содержит не более $(i-1)$ рёбер, то по индукционному предположению $d[v]$ уже равно длине этого пути.

Если $p$ содержит ровно $i$ рёбер, то пусть $u$ — предпоследняя вершина на этом пути. Тогда путь от $s$ до $u$ содержит $(i-1)$ ребро и по индукционному предположению $d[u]$ равно длине этого пути. При обработке ребра $(u, v)$ на $i$-й итерации алгоритм установит $d[v] = \min(d[v], d[u] + w(u, v))$, что даст длину кратчайшего пути до $v$, содержащего не более $i$ рёбер.

Поскольку в графе без отрицательных циклов кратчайший путь содержит не более $(|V| - 1)$ рёбер, после $(|V| - 1)$ итераций внешнего цикла $d[v]$ будет равно длине кратчайшего пути от $s$ до $v$ для всех вершин $v$.

Дополнительная итерация алгоритма используется для проверки наличия отрицательных циклов. Если после $(|V| - 1)$ итераций какое-либо расстояние можно уменьшить, то в графе существует отрицательный цикл, достижимый из $s$.
$\blacksquare$


In [ ]:
def bellman_ford(graph: Dict[int, List[Tuple[int, int]]], start: int, num_vertices: int) -> Tuple[Dict[int, int], bool]:
    """
    Реализация алгоритма Беллмана-Форда для поиска кратчайших путей от начальной вершины.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, вес ребра)
        start: начальная вершина
        num_vertices: общее количество вершин в графе

    Returns:
        кортеж (словарь расстояний, флаг наличия отрицательного цикла)
    """
    # Инициализация расстояний
    distances = {vertex: float('infinity') for vertex in range(num_vertices)}
    distances[start] = 0

    # Преобразуем граф в список рёбер для удобства
    edges = []
    for u in graph:
        for v, weight in graph[u]:
            edges.append((u, v, weight))

    # Основной алгоритм: релаксация всех рёбер |V| - 1 раз
    for _ in range(num_vertices - 1):
        for u, v, weight in edges:
            if distances[u] != float('infinity') and distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight

    # Проверка на наличие отрицательных циклов
    has_negative_cycle = False
    for u, v, weight in edges:
        if distances[u] != float('infinity') and distances[u] + weight < distances[v]:
            has_negative_cycle = True
            break

    return distances, has_negative_cycle

# Пример использования
graph = {
    0: [(1, 4), (2, 1)],
    1: [(3, 1)],
    2: [(1, 2), (3, 5)],
    3: []
}

distances, has_negative_cycle = bellman_ford(graph, 0, 4)
print(distances)  # {0: 0, 1: 3, 2: 1, 3: 4}
print(f"Граф содержит отрицательный цикл: {has_negative_cycle}")  # False

# Пример с отрицательным циклом
graph_with_negative_cycle = {
    0: [(1, 1)],
    1: [(2, 2)],
    2: [(3, 3)],
    3: [(1, -7)]
}

distances, has_negative_cycle = bellman_ford(graph_with_negative_cycle, 0, 4)
print(f"Граф содержит отрицательный цикл: {has_negative_cycle}")  # True


**Теорема:** Временная сложность алгоритма Беллмана-Форда составляет $O(|V| \cdot |E|)$.

$\square$
Алгоритм Беллмана-Форда состоит из следующих операций:
- Инициализация расстояний: $O(|V|)$
- Внешний цикл выполняется $|V|$ раз
- Внутренний цикл перебирает все рёбра, что требует $O(|E|)$ операций
- Проверка на отрицательные циклы: $O(|E|)$

Итоговая сложность: $O(|V| + |V| \cdot |E| + |E|) = O(|V| \cdot |E|)$
$\blacksquare$


**Замечание:** Алгоритм Беллмана-Форда имеет большую временную сложность, чем алгоритм Дейкстры, но может работать с графами, содержащими рёбра с отрицательными весами.


## Алгоритм Флойда-Уоршелла


**Идея:** Алгоритм Флойда-Уоршелла решает задачу поиска кратчайших путей между всеми парами вершин в графе с произвольными весами рёбер. Алгоритм использует динамическое программирование для постепенного улучшения оценок расстояний.

**Теорема:** Алгоритм Флойда-Уоршелла корректно находит кратчайшие пути между всеми парами вершин в графе с произвольными весами рёбер, если в графе нет отрицательных циклов.

$\square$
Докажем корректность алгоритма Флойда-Уоршелла.

Пусть $d^k[i][j]$ — длина кратчайшего пути из вершины $i$ в вершину $j$, использующего в качестве промежуточных только вершины из множества $\{1, 2, \ldots, k\}$.

Утверждение: После завершения $k$-й итерации внешнего цикла значение $d[i][j]$ равно $d^k[i][j]$ для всех пар вершин $(i, j)$.

Докажем это утверждение индукцией по $k$.

База индукции ($k = 0$): После инициализации $d[i][j]$ равно весу ребра $(i, j)$, если оно существует, или $\infty$, если такого ребра нет. Это соответствует $d^0[i][j]$, так как путь без промежуточных вершин — это просто прямое ребро.

Индукционный переход: Предположим, что утверждение верно для $(k-1)$-й итерации. Докажем, что оно верно для $k$-й итерации.

Для любой пары вершин $(i, j)$ кратчайший путь, использующий только вершины из множества $\{1, 2, \ldots, k\}$, может быть одним из двух типов:
1. Путь, не проходящий через вершину $k$. В этом случае $d^k[i][j] = d^{k-1}[i][j]$.
2. Путь, проходящий через вершину $k$. В этом случае $d^k[i][j] = d^{k-1}[i][k] + d^{k-1}[k][j]$.

Алгоритм выбирает минимум из этих двух вариантов:
$d^k[i][j] = \min(d^{k-1}[i][j], d^{k-1}[i][k] + d^{k-1}[k][j])$

Это в точности соответствует формуле обновления в алгоритме Флойда-Уоршелла.

После завершения $|V|$-й итерации внешнего цикла значение $d[i][j]$ будет равно $d^{|V|}[i][j]$, что является длиной кратчайшего пути из $i$ в $j$, использующего любые вершины графа в качестве промежуточных.

Если в графе нет отрицательных циклов, то эти значения будут корректными длинами кратчайших путей между всеми парами вершин.
$\blacksquare$


In [ ]:
def floyd_warshall(graph: List[List[float]], num_vertices: int) -> List[List[float]]:
    """
    Реализация алгоритма Флойда-Уоршелла для поиска кратчайших путей между всеми парами вершин.

    Args:
        graph: матрица смежности, где graph[i][j] - вес ребра (i, j) или float('infinity'), если ребра нет
        num_vertices: количество вершин в графе

    Returns:
        матрица кратчайших расстояний между всеми парами вершин
    """
    # Создаем копию матрицы смежности
    dist = [row[:] for row in graph]

    # Основной алгоритм
    for k in range(num_vertices):
        for i in range(num_vertices):
            for j in range(num_vertices):
                if dist[i][k] != float('infinity') and dist[k][j] != float('infinity'):
                    dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])

    return dist

# Пример использования
# Создаем матрицу смежности для графа
INF = float('infinity')
graph = [
    [0, 4, 1, INF],
    [INF, 0, INF, 1],
    [INF, 2, 0, 5],
    [INF, INF, INF, 0]
]

distances = floyd_warshall(graph, 4)
for row in distances:
    print([int(d) if d != INF else "INF" for d in row])
# Ожидаемый результат:
# [0, 3, 1, 4]
# [INF, 0, INF, 1]
# [INF, 2, 0, 3]
# [INF, INF, INF, 0]


**Теорема:** Временная сложность алгоритма Флойда-Уоршелла составляет $O(|V|^3)$.

$\square$
Алгоритм Флойда-Уоршелла состоит из трёх вложенных циклов, каждый из которых выполняется $|V|$ раз. Внутри самого внутреннего цикла выполняется константное число операций.

Итоговая сложность: $O(|V| \cdot |V| \cdot |V|) = O(|V|^3)$
$\blacksquare$


**Замечание:** Алгоритм Флойда-Уоршелла имеет кубическую сложность от числа вершин, что делает его неэффективным для больших графов. Однако он прост в реализации и может работать с графами, содержащими рёбра с отрицательными весами (при отсутствии отрицательных циклов).


## Алгоритм A*


**Идея:** Алгоритм A* является информированным алгоритмом поиска пути и используется для нахождения кратчайшего пути между двумя вершинами. Он использует эвристическую функцию для оценки расстояния до целевой вершины, что позволяет сократить пространство поиска.

**Определение:** Эвристическая функция $h(v)$ оценивает расстояние от вершины $v$ до целевой вершины $t$. Функция $h$ называется допустимой, если для любой вершины $v$ выполняется $h(v) \leq \delta(v, t)$.

**Определение:** Эвристическая функция $h(v)$ называется монотонной (или согласованной), если для любого ребра $(u, v)$ выполняется $h(u) \leq w(u, v) + h(v)$.

**Теорема:** Если эвристическая функция $h$ является допустимой и монотонной, то алгоритм A* находит кратчайший путь от начальной вершины до целевой.

$\square$
Докажем, что алгоритм A* с допустимой и монотонной эвристикой находит кратчайший путь.

Пусть $g(v)$ — длина пути от начальной вершины $s$ до вершины $v$, найденного алгоритмом A*, а $f(v) = g(v) + h(v)$ — оценка полной длины пути от $s$ до $t$ через $v$.

Утверждение 1: Если $h$ монотонна, то значение $g(v)$ для каждой извлеченной из очереди вершины $v$ является длиной кратчайшего пути от $s$ до $v$.

Докажем это утверждение. Предположим противное: существует вершина $v$, для которой $g(v) > \delta(s, v)$ в момент её извлечения из очереди.

Пусть $p = \langle s, v_1, v_2, \ldots, v_k, v \rangle$ — кратчайший путь от $s$ до $v$. Пусть $v_i$ — первая вершина на этом пути, которая ещё находится в очереди (или не была добавлена в неё) в момент извлечения $v$. Тогда $v_{i-1}$ уже была извлечена из очереди, и по индукционному предположению $g(v_{i-1}) = \delta(s, v_{i-1})$.

При обработке $v_{i-1}$ алгоритм должен был обновить значение $g(v_i)$ до $g(v_{i-1}) + w(v_{i-1}, v_i) = \delta(s, v_{i-1}) + w(v_{i-1}, v_i) = \delta(s, v_i)$.

Поскольку $h$ монотонна, для любой вершины $u$ на пути от $v_i$ до $v$ выполняется $f(u) \leq f(v)$. Следовательно, $f(v_i) \leq f(v)$.

Но алгоритм A* выбирает вершину с минимальным значением $f$, поэтому $v_i$ должна была быть извлечена из очереди раньше $v$. Противоречие.

Утверждение 2: Когда алгоритм A* извлекает целевую вершину $t$ из очереди, значение $g(t)$ равно длине кратчайшего пути от $s$ до $t$.

Это следует непосредственно из Утверждения 1.
$\blacksquare$


In [ ]:
import heapq
from typing import Dict, List, Tuple, Callable

def a_star(graph: Dict[int, List[Tuple[int, int]]], start: int, goal: int, 
           heuristic: Callable[[int], float]) -> Tuple[List[int], float]:
    """
    Реализация алгоритма A* для поиска кратчайшего пути между двумя вершинами.

    Args:
        graph: словарь смежности, где ключи - вершины, значения - списки пар (соседняя вершина, вес ребра)
        start: начальная вершина
        goal: целевая вершина
        heuristic: эвристическая функция, оценивающая расстояние до целевой вершины

    Returns:
        кортеж (путь, длина пути) или ([], float('infinity')), если путь не найден
    """
    # Открытый список (очередь с приоритетами)
    open_set = [(0 + heuristic(start), 0, start, [start])]

    # Закрытый список (посещенные вершины)
    closed_set = set()

    # Словарь для хранения лучших известных расстояний до вершин
    g_score = {vertex: float('infinity') for vertex in graph}
    g_score[start] = 0

    while open_set:
        # Извлекаем вершину с минимальным f-значением
        f, current_distance, current_vertex, path = heapq.heappop(open_set)

        # Если достигли целевой вершины, возвращаем путь и расстояние
        if current_vertex == goal:
            return path, current_distance

        # Если вершина уже обработана, пропускаем
        if current_vertex in closed_set:
            continue

        # Добавляем вершину в закрытый список
        closed_set.add(current_vertex)

        # Обрабатываем соседние вершины
        for neighbor, weight in graph[current_vertex]:
            # Если вершина уже обработана, пропускаем
            if neighbor in closed_set:
                continue

            # Вычисляем новое расстояние до соседа
            tentative_g_score = current_distance + weight

            # Если найден более короткий путь, обновляем
            if tentative_g_score < g_score[neighbor]:
                # Обновляем путь и расстояние
                new_path = path + [neighbor]
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor)

                # Добавляем в очередь
                heapq.heappush(open_set, (f_score, tentative_g_score, neighbor, new_path))

    # Если путь не найден
    return [], float('infinity')

# Пример использования
# Определяем эвристическую функцию (манхэттенское расстояние)
def manhattan_distance(vertex: int) -> float:
    # В этом примере вершины представлены как числа
    # В реальном приложении здесь может быть расчет расстояния
    # между координатами вершин
    return abs(vertex - 3)  # Предполагаем, что цель - вершина 3

graph = {
    0: [(1, 4), (2, 1)],
    1: [(3, 1)],
    2: [(1, 2), (3, 5)],
    3: []
}

path, distance = a_star(graph, 0, 3, manhattan_distance)
print(f"Путь: {path}")  # [0, 2, 1, 3]
print(f"Расстояние: {distance}")  # 4


**Теорема:** Временная сложность алгоритма A* в худшем случае составляет $O(|E| \log |V|)$, где $|E|$ — количество рёбер в графе, а $|V|$ — количество вершин.

$\square$
В худшем случае алгоритм A* может исследовать все рёбра графа. Операции с очередью с приоритетами требуют $O(\log |V|)$ времени, где $|V|$ — количество вершин.

Итоговая сложность: $O(|E| \log |V|)$
$\blacksquare$


**Замечание:** Эффективность алгоритма A* сильно зависит от выбора эвристической функции. При использовании тривиальной эвристики $h(v) = 0$ алгоритм A* вырождается в алгоритм Дейкстры.


## Заключение


В данном разделе мы рассмотрели основные алгоритмы поиска кратчайших путей в графах:

1. **Алгоритм Дейкстры**
   - Применим для графов с неотрицательными весами рёбер
   - Временная сложность: $O((|V| + |E|) \log |V|)$ с использованием бинарной кучи
   - Использует жадную стратегию выбора вершины с минимальным текущим расстоянием

2. **Алгоритм Беллмана-Форда**
   - Применим для графов с произвольными весами рёбер, включая отрицательные
   - Временная сложность: $O(|V| \cdot |E|)$
   - Может определить наличие отрицательных циклов

3. **Алгоритм Флойда-Уоршелла**
   - Находит кратчайшие пути между всеми парами вершин
   - Временная сложность: $O(|V|^3)$
   - Использует динамическое программирование

4. **Алгоритм A***
   - Использует эвристическую функцию для оптимизации поиска
   - Временная сложность: $O(|E| \log |V|)$ в худшем случае
   - Эффективен при наличии хорошей эвристики

**Сравнение алгоритмов:**

| Алгоритм | Временная сложность | Применимость | Особенности |
|----------|---------------------|--------------|-------------|
| Дейкстра | $O((|V| + |E|) \log |V|)$ | Графы с неотрицательными весами | Эффективен для разреженных графов |
| Беллман-Форд | $O(|V| \cdot |E|)$ | Графы с произвольными весами | Обнаруживает отрицательные циклы |
| Флойд-Уоршелл | $O(|V|^3)$ | Графы с произвольными весами | Находит все пары кратчайших путей |
| A* | $O(|E| \log |V|)$ | Зависит от эвристики | Оптимизирует поиск с помощью эвристики |

**Выбор алгоритма:**
- Для графов с неотрицательными весами и поиска путей из одной вершины: **Дейкстра**
- Для графов с отрицательными весами и поиска путей из одной вершины: **Беллман-Форд**
- Для поиска путей между всеми парами вершин: **Флойд-Уоршелл**
- Для поиска пути между двумя конкретными вершинами с использованием дополнительной информации: **A***

Алгоритмы поиска кратчайших путей имеют широкое применение в различных областях:
- Навигационные системы и маршрутизация
- Сетевые протоколы (например, OSPF, BGP)
- Логистика и транспортные задачи
- Компьютерные игры (поиск пути для персонажей)
- Робототехника (планирование движения)
